In [7]:
!wget https://raw.githubusercontent.com/omniscientoctopus/Physics-Informed-Neural-Networks/7d26e55cc665b79b652b226860177fa4defe9190/PyTorch/Burgers\'%20Equation/Data/burgers_shock_mu_01_pi.mat

--2025-01-19 11:22:19--  https://raw.githubusercontent.com/omniscientoctopus/Physics-Informed-Neural-Networks/7d26e55cc665b79b652b226860177fa4defe9190/PyTorch/Burgers'%20Equation/Data/burgers_shock_mu_01_pi.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 157214 (154K) [application/octet-stream]
Saving to: ‘burgers_shock_mu_01_pi.mat’

burgers_shock_mu_01 100%[===================>] 153.53K  --.-KB/s    in 0.03s   

2025-01-19 11:22:19 (4.68 MB/s) - ‘burgers_shock_mu_01_pi.mat’ saved [157214/157214]



# pinn class

In [26]:
import torch
import torch.nn as nn
import numpy as np
from scipy.io import loadmat
from tqdm import tqdm

class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(input_size if i == 0 else hidden_size, hidden_size) if i %2 == 0 else nn.Tanh() for i in range(20)])
        self.layers.append(nn.Linear(hidden_size, output_size))
        self.loss = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    # ic and bc is of shape [x, 3]. first two columns is x and t, last column is u.
    # predict using [:2], calc loss using [2]
    def loss_fn(self, cond):
        u_pred = self.forward(cond[:, :2])
        return self.loss(u_pred, cond[:, 2])

    def residual_loss(self, x_f_train, nu, fhat):
        x = x_f_train[:, 0]
        t = x_f_train[:, 1]
        g = x_f_train.clone()
        u_pred = self.forward(g)
        u_t = torch.autograd.grad(u_pred, t, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]
        u_x = torch.autograd.grad(u_pred, x, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        return self.loss(u_t + u_pred * u_x - (nu * u_xx), fhat)

    def total_loss(self, x_f_train, ic_bc, nu, fhat):
        return self.loss_fn(ic_bc) + self.residual_loss(x_f_train, nu, fhat)

    def train_model(self, x_f_train, ic_bc, nu, epochs=1000, lr=0.001):
        fhat = torch.zeros(x_f_train.shape[0], 1)
        for epoch in tqdm(range(epochs)):
            self.optimizer.zero_grad()
            loss = self.total_loss(x_f_train, ic_bc, nu, fhat)
            loss.backward()
            self.optimizer.step()


model = PINN(input_size=2, hidden_size=20, output_size=1)
print(model)

PINN(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Tanh()
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): Tanh()
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): Tanh()
    (8): Linear(in_features=20, out_features=20, bias=True)
    (9): Tanh()
    (10): Linear(in_features=20, out_features=20, bias=True)
    (11): Tanh()
    (12): Linear(in_features=20, out_features=20, bias=True)
    (13): Tanh()
    (14): Linear(in_features=20, out_features=20, bias=True)
    (15): Tanh()
    (16): Linear(in_features=20, out_features=20, bias=True)
    (17): Tanh()
    (18): Linear(in_features=20, out_features=20, bias=True)
    (19): Tanh()
    (20): Linear(in_features=20, out_features=1, bias=True)
  )
  (loss): MSELoss()
)


# load data

In [3]:
data = loadmat("burgers_shock_mu_01_pi.mat")
u = data["usol"]
x = data["x"]
t = data["t"]
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.float32)
u = torch.tensor(u, dtype=torch.float32)
# form a meshgrid, where X contains all the x points from -1 to 1, 100 times for 100 time steps.
# T contains all 100 time steps from 0 to 1, 256 times for 256 x points
X, T = np.meshgrid(x, t)
X.shape, T.shape

((100, 256), (100, 256))

# initial and boundary conditions

In [4]:
# initial conditions, x at t=0. shape of 256, 1
xic, tic, uic = X[0, :], T[0, :], u[:, 0]
# bc1, x=1, t goes from 0 to 1. shape of 100, 1
xbc1, tbc1, ubc1 = X[:, 0], T[:, 0], u[0, :]
# bc2, x=-1, t goes from 0 to 1. shape of 100, 1
xbc2, tbc2, ubc2 = X[:, -1], T[:, -1], u[-1, :]

# Convert xic, tic, xbc1, tbc1, xbc2, and tbc2 to PyTorch tensors
xic = torch.tensor(xic, dtype=torch.float32)
tic = torch.tensor(tic, dtype=torch.float32)
xbc1 = torch.tensor(xbc1, dtype=torch.float32)
tbc1 = torch.tensor(tbc1, dtype=torch.float32)
xbc2 = torch.tensor(xbc2, dtype=torch.float32)
tbc2 = torch.tensor(tbc2, dtype=torch.float32)

# stack em all together so we can pass it to the loss function. Shape of [n, 3]
ic = torch.stack([xic, tic, uic], dim=1)
bc1 = torch.stack([xbc1, tbc1, ubc1], dim=1)
bc2 = torch.stack([xbc2, tbc2, ubc2], dim=1)
bc = torch.vstack([bc1, bc2])

ic.shape, bc.shape

(torch.Size([256, 3]), torch.Size([200, 3]))

# collocation points

In [17]:
from pyDOE import lhs

X_u_test = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
lb = X_u_test[0]  
ub = X_u_test[-1]
lb, ub 
idx = np.random.choice(456, 100, replace=False)
ic_bc_train = np.vstack([ic, bc])[idx, :]
X_f_train = lb + (ub - lb) * lhs(2, 10000)
X_f_train = np.vstack((X_f_train, ic_bc_train[:, :2]))
X_f_train.shape, ic_bc_train.shape


((10100, 2), (100, 3))

In [27]:
device = torch.device("cuda")
X_f_train = torch.tensor(X_f_train, dtype=torch.float32, device=device)
ic_bc_train = torch.tensor(ic_bc_train, dtype=torch.float32, device=device)
model = model.to(device)


/tmp/ipykernel_5546/3763497953.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_f_train = torch.tensor(X_f_train, dtype=torch.float32, device=device)
/tmp/ipykernel_5546/3763497953.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ic_bc_train = torch.tensor(ic_bc_train, dtype=torch.float32, device=device)


In [28]:
model.train_model(X_f_train, ic_bc_train, 0.01, 1000, 0.001)

  0%|          | 0/1000 [00:00<?, ?it/s]/home/shusrith/projects/torch/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: One of the differentiated Tensors does not require grad